# Магические методы


### `__init()__`

In [2]:
# __init()__
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    def get_email_data(self):
        return {
            'name': self.name,
            'email': self.email
        }
jane = User('Jane Doe', 'janedoe@example.com')

print(jane.get_email_data())

{'name': 'Jane Doe', 'email': 'janedoe@example.com'}


### `__new__`

In [4]:
# __new__()
class Singleton:
    instance = None
    
    def __new__(cls):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance
    
a = Singleton()
b = Singleton()

print(a is b)
        

True


### `__str__`
Определяет человеко-читаемое описание нашего класса

In [6]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    def __str__(self):
        return '{} <{}>'.format(self.name, self.email)

jane = User('Jane Doe', 'janedoe@example.com')
print(jane)



Jane Doe <janedoe@example.com>


### `__hash__, __eq__`
Определяют как сравниваются наши объекты и что происходит при вызове функции hash()

In [22]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    def __hash__(self):
        return hash(self.email)
        #return hash(self.name)
    def __eq__(self, obj):
        return self.email ==  obj.email
    
jane = User('Jane Doe', 'jdoe@example.com')
joe = User('Joe Doe', 'jdoe@example.com')

print(jane == joe)

print(hash(jane)) # hash is the same
print(hash(joe))

# если мы попытаемся создать словарь из этих экземпляров то ключ будет только один, так как hash одинаковый так как он генерируется на основе email, который одинаков
# (попробуй вернуть из hash уникальное значение)
user_email_map = {user: user.name for user in [jane, joe]}
print(user_email_map)

True
-7004153980191167432
-7004153980191167432
{<__main__.User object at 0x107be4150>: 'Joe Doe'}


### `__getattr__, __getattribute__, `
### `__setattr__, __delattr__`

In [57]:
class Researcher:
    def __getattr__(self, name): # вызывется тогда, когда не найден атрибут
        return 'Nothing found :('
    def __getattribute__(self, name): # вызывается всегда, когда мы обращаемся к атрибуту, существующему или нет; можно использовать для логирования обращений
        return 'nope'
    
obj = Researcher()
print(obj.attr)
print(obj.method)



nope
nope


In [59]:
class Researcher:
    # вызывется тогда, когда не найден атрибут
    def __getattr__(self, name): 
        return 'Nothing found :('
    # вызывается всегда, когда мы обращаемся к атрибуту, существующему или нет; можно использовать для логирования обращений
    def __getattribute__(self, name): 
        print('Looking for {}'.format(name))
         #return object.__getattribute__(self, name)
        return object.__getattribute__(self, name)
    
obj = Researcher()
print(obj.attr)
print(obj.method)



Looking for attr
Nothing found :(
Looking for method
Nothing found :(


In [42]:
# __settattr__ вызывается всегда при присваивании значения атрибуту
class Ignorant:
    def __setattr__(self, name, value):
        print("Setting attribute... Not gonna set '{}'!".format(name))
        
obj = Ignorant()
obj.math = True

Setting attribute... Not gonna set 'math'!


In [43]:
# __delattr__ вызывается когда мы пытаемся удалить атрибут
# используется когда мы хотим переопределить поведение удалятора, например, каскадно удалить другие объекты
class Polite:
    def __delattr__(self, name):
        value = getattr(self, name)
        print(f'Goodbye {name}, you were {value}!')
        object.__delattr__(self, name)

obj = Polite()
obj.attr = 10
del obj.attr



Godbye attr, you were 10!


### `__call__`

In [60]:
# определяет поведение когда наш класс вызывается
# например мы можем определить класс Loger который будет в качестве декоратора вызываться при вызове какой-либо функции для логирования определенных действий
import datetime

class Logger:
    def __init__(self, filename):
        self.filename = filename
        
    def __call__(self, func):
        with open(self.filename, 'w') as f:
            f.write('Class Logger was called at {}'.format(datetime.datetime.now()))
        return func
    
logger = Logger('/Users/Andrew/Desktop/log1.txt')

@logger
def completely_useless_function():
    pass

completely_useless_function()

with open('/Users/Andrew/Desktop/log1.txt', 'r') as f:
    print(f.read())

Class Logger was called at 2020-07-10 18:41:09.230562


В этом примере, к сожалению, допущена ошибка. При определении метода __call__, мы меняем поведение объекта класса при его вызове с помощью круглых скобочек. В данном примере объект logger вызывается, когда используется как декоратор, то есть при определении "бессмысленной" функции. Таким образом мы записываем в файл не вызов функции, а применение объекта в качестве декоратора. Для логирования вызовов функции код должен был быть примерно такой:

In [51]:
class Logger:
    def __init__(self, filename):
        self.filename = filename
        
    def __call__(self, func):
        def wrapped(*args, **kwargs):
            with open(self.filename, 'w') as f:
                f.write('Logging function call...')
        
            return func(*args, *kwargs)
        return wrapped
    
logger = Logger('/Users/Andrew/Desktop/log1.txt')

@logger
def completely_useless_function():
    pass

completely_useless_function()

with open('/Users/Andrew/Desktop/log1.txt', 'r') as f:
    print(f.read())

Logging function call...


### `__add__`

In [54]:
import random

class NoisyInt:
    def __init__(self, value):
        self.value = value
    def __add__(self, obj):
        noise = random.uniform(-1, 1)
        return self.value + obj.value + noise
    
a = NoisyInt(10)
b = NoisyInt(20)

for _ in range(3):
    print(a + b)

29.948661633259537
29.58237378221851
29.92500999661754


### `__getitem__, __setitem__`

In [56]:
# в Паскале листы индексируются с 1. Создадим класс который имитирует их
class PascalList:
    def __init__(self, original_list = None):
        self.container = original_list or []
        
    def __getitem__(self, index):
        return self.container[index - 1]
    def __setitem__(self, index, value):
        self.container[index - 1] = value
    def __str__(self):
        return self.container.__str__()
    
numbers = PascalList([1,2,3,4,5])

print(numbers[1])
print(numbers[5])

1
5


# Итераторы


In [61]:
for number in range(5):
    print(number)

0
1
2
3
4


In [63]:
for letter in 'python':
    print('{0} - {1}'.format(letter, ord(letter)))

p - 112
y - 121
t - 116
h - 104
o - 111
n - 110


In [65]:
# можно создать свой итератор
iterator = iter([1,2,3])
print(next(iterator))
print(next(iterator))
print(next(iterator))

1
2
3


In [67]:
iterator = iter(['Andrei','Maxim','Sergei'])
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))

Andrei
Maxim
Sergei


StopIteration: 

В Python можно реализовать свой итератор написав класс с соответсвующими методами

In [68]:
class SquareIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
    def __iter__(self):
        return self
    def __next__(self):  # определяет, что возвращается при следующей итерации
        if self.current >= self.end:
            raise StopIteration
        result = self.current ** 2
        self.current += 1
        return result
    
for num in SquareIterator(1,4):
    print(num)
    

1
4
9


Можно определить свой класс-итератор и без определения iter и next

In [69]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
    def __getitem__(self, index):
        return self.obj[index]
    
for letter in IndexIterable('str'):
    print(letter)

s
t
r


# Контекстные менеджеры

In [ ]:
# простейший контекстный менеджер
with open('access_log.log', 'a') as f:
    f.write('New Access')

In [1]:
# Для определения своего контекстного менеджера необходимо определить класс с особыми магическими методами
class open_file: # начитается малой буквы так как это контекстный менеджер
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    def __enter__(self): # определяет что происходит в начале
        return self.f  # то что записывается в переменную file (смотри ниже)
    def __exit__(self, *args): # определяет что присходит в конце
        self.f.close()
        
with open_file('/Users/Andrew/Documents/python/working/data/test.log', 'w') as file:
    file.write('Inside `open_file` context manger')
    
with open_file('/Users/Andrew/Documents/python/working/data/test.log', 'r') as file:
    print(file.read())

Inside `open_file` context manger


Еще одна важная особенность контекстных менеджеров - они позволяют управлять искючениями которые произошли внутри блока

In [6]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
    def __enter__(self):
        return
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print('Nothing bad happend.. continue.')
            return True
    

with suppress_exception(ZeroDivisionError):
    really_big_number = 1/0
    print(really_big_number)


Nothing bad happend.. continue.


Такой контекстный менеджер уже есть в стандартной библиотеке

In [7]:
import contextlib

with contextlib.suppress(ValueError):
    raise ValueError

### Пример: напишем контекстный менеджер который считает и выводит время, проведенное внутри его

In [20]:
import time

class timer():
    def __init__(self):
        self.start = time.time()
    def current_time(self):
        return time.time() - self.start
    
    def __enter__(self):
        return self
    def __exit__(self, *args):
        print('Elapsed time: {}'.format(self.current_time()))
    
with timer() as t:
    #print('123')
    time.sleep(1)
    print('Current time: {}'.format(t.current_time()))
    time.sleep(1)

Current time: 1.001657247543335
Elapsed time: 2.0059990882873535


# Programming Assignment: Файл с магическими методами
В этом задании вам нужно создать интерфейс для работы с файлами. Интерфейс должен предоставлять следующие возможности по работе с файлами:

- чтение из файла, метод read возвращает строку с текущим содержанием файла

- запись в файл, метод write принимает в качестве аргумента строку с новым содержанием файла

- сложение объектов типа File, результатом сложения является объект класса File, при этом создается новый файл и файловый объект, в котором содержимое второго файла добавляется к содержимому первого файла. Новый файл должен создаваться в директории, полученной с помощью функции tempfile.gettempdir (https://docs.python.org/3/library/tempfile.html). Для получения нового пути можно использовать os.path.join (https://docs.python.org/3/library/os.path.html#os.path.join).

- возвращать в качестве строкового представления объекта класса File полный путь до файла

- поддерживать протокол итерации, причем итерация проходит по строкам файла

При создании экземпляра класса File в конструктор передается полный путь до файла на файловой системе. Если файла с таким путем не существует, он должен быть создан при инициализации.



In [75]:
import tempfile
import os
tempfile.tempdir = '/Users/Andrew/tmp'
tempfile.gettempdir()

class File:
    def __init__(self, path):
        self.path = path
        
        #print(os.path.exists(self.path))
        if not os.path.exists(self.path):
            with open(self.path, 'w') as f:
                f.write('')
    def read(self):
        with open(self.path, 'r') as f:
                result = f.read()
        #try:
        #    with open(self.path, 'r') as f:
        #        result = f.read()
        #except FileNotFoundError:
        #    print('Sorry, there is no such file')
        #    result = ''
        return result
    def write(self, string):
        with open(self.path, 'w') as f:
            f.write(string)
    def __add__(self, obj):
        new_path = os.path.join(tempfile.gettempdir(), 'file3.txt')
        file_obj_new = File(new_path)
        file_obj_new.write(self.read() + obj.read())  
        return file_obj_new
    def __str__(self):
        return self.path
    def __iter__(self):
        self.file = open(self.path, 'r')
        #with open(self.path, 'r') as f:
        #       list_of_lines = f.readlines()
        #return list_of_lines
        return self
    def __next__(self):
        line = self.file.readline()
        if line == '':
            self.file.close()
            raise StopIteration
        return line
    


### Testing

In [77]:
import os.path

path = '/Users/Andrew/tmp/file'
os.path.exists(path)

# 1
file_obj = File(path)
print('1. file_obj.read():', file_obj.read())

# 2
print('2. os.path.exists(path): ', os.path.exists(path))

# 3
file_obj.write('some text')
print('3. file_obj.read(): ', file_obj.read())

# 4
file_obj_1 = File(path + '_1')
file_obj_2 = File(path + '_2')
file_obj_1.write('line 1 \n')
file_obj_2.write('line 2 \n')
file_obj_new = file_obj_1 + file_obj_2
isinstance(file_obj_new, File)
print('4. file_obj_new: ', file_obj_new)
print('5. file_obj_new.read(): ', file_obj_new.read())

# 6
print('6. reading lines iterating through file')
for line in file_obj_new:
    print(line)

1. file_obj.read(): some text
2. os.path.exists(path):  True
3. file_obj.read():  some text
4. file_obj_new:  /Users/Andrew/tmp/file3.txt
5. file_obj_new.read():  line 1 
line 2 

6. reading lines iterating through file
line 1 

line 2 



In [69]:
f = open('/Users/Andrew/tmp/file3.txt', 'r')
print(f.readline())
print(f.readline())
print(f.readline())

f.

line 1 

line 2 




# Дескрипторы
С помощью дескрипторов в Python реализована практически вся магия при работе с объектами, классами и методами. Чтобы определить свой собственный дескриптор, нужно определить класс. методы __get__, __set__ или __delete__. После этого мы можем создать какой-то новый класс и в атрибут этого класса записать объект типа дескриптор.

Документация: https://docs.python.org/3/howto/descriptor.html

In [ ]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
    def __set__(self, obj, value):
        print('set')
    def __delete__(self, obj):
        print('delete')
        
class Class:
    attr = Descriptor()
    
instance = Class()

Таким образом, Python позволяет вам переопределять поведение при доступе к атрибуту. Это очень мощная концепция, мощный механизм, который позволяет вам незаметно от пользователя определять различные поведения в ваших классах. 

In [7]:
class Value:
    def __init__(self):
        self.value = None
    @staticmethod
    def _prepare_value(value):
        return value * 10
    def __get__(self, obj, obj_type): # obj - объект который вызвал дескриптор, obj_type - его тип
        print('__get__', obj)
        print('__get__', obj_type)
        return self.value
    def __set__(self, obj, value):
        self.value = self._prepare_value(value)
        #print('__set__', obj)
        


class Class:
    attr = Value()
    
instance = Class()
instance.attr = 10
print(instance.attr)

__get__ <__main__.Class object at 0x1065cf310>
__get__ <class '__main__.Class'>
100


### Пример: написать декскриптор, который пишет в файл все присваиваемые ему значения

In [14]:
import datetime

class ImportantValue:
    def __init__(self, amount):
        self.amount = amount
    def __get__(self, obj, obj_type):
        pass
    def __set__(self, obj, value):    
        with open('data/log.txt', 'w') as f:
            f.write('{} - {}\n'.format(datetime.datetime.now(), str(value)))
        self.amount = value

class Account:
    amount = ImportantValue(100)
    
bobs_account = Account()
bobs_account.amount = 150 # here writing to log file should occure

with open('data/log.txt', 'r') as f:
    print(f.read())

2020-07-13 18:21:50.638374 - 150



### Функции и методы в Python реализованы с помощью дескрипторов

In [15]:
class Class:
    def method(self):
        pass
    
obj = Class()

print(obj.method) # returns bound method те это метод привязанный к какому-то объекту
print(Class.method) # returns unbound method

<bound method Class.method of <__main__.Class object at 0x106b5be10>>
<function Class.method at 0x106b58dd0>


В bound метод по умолчанию передаётся объект, с которым вызван метод. И именно он и записывается в атрибут self метода класса. 
Один и тот же метод возвращает разные объекты в зависимости от того как к нему образаются. Это и есть поведение дескриптора.

Вам уже знаком декоратор property, который позволяет вам использовать функцию как атрибут класса (без скобочек). В данном случае мы можем определить property full_name, который на самом деле хоть и является функцией, которая возвращает строчку, используется потом так же, как и обычный атрибут, то есть без вызова скобочек. В данном случае у нас класс User, у нас first_name и last_name, и full_name возвращает, очевидно, полное имя. При вызове full_name от объекта у нас вызывается функция full_name. Однако если мы пытаемся обратиться к full_name от класса, у нас получится объект типа property. На самом деле, property реализовано с помощью дескрипторов, потому что разное поведение в зависимости от того, как у нас вызывается этот объект. 

In [16]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
amy = User('Amy', 'Janes')

print(amy.full_name)
print(User.full_name)

Amy Janes


И мы можем написать свой собственный класс property, который будет эмулировать поведение стандартного property. Для этого нам нужно сохранить функцию, которую property получает, потому что property — это декоратор, он получает функцию. И когда мы обращаемся к нашему объекту, если он вызван от класса, мы просто возвращаем самого себя, а если у нас вызван наш атрибут с объектом, то мы возвращаем соответствующий getter, вызываем функцию. Таким образом, мы можем определить класс и использовать как стандартный декоратор property, так и новый только что созданный. В двух видах можем просто его использовать как декоратор с помощью синтаксического сахара, можем использовать как вызов функции. И окажется, что они работают идентично, потому что на самом деле property реализован именно с помощью дескриптора. Точно так же реализованы StaticMethod и ClassMethod.

In [17]:
# property делает возможным обращение к методу без скобок, как атрибуту

class Property:
    def __init__(self, getter):
        self.getter = getter
        
    def __get__(self, obj, obj_type = None):
        if obj is None:  # вызов метода классом
            return self
        return self.getter(obj) # вызов метода экземпляром класса
    
class Class:
    @property
    def original(self):
        return 'original'
    @Property
    def custom_sugar(self):
        return 'custom sugar'
    def custom_pure(self):
        return 'custom pure'
    custom_pure = Property(custom_pure)
    
obj = Class()
print(obj.original)
print(obj.custom_sugar)
print(obj.custom_pure)

original
custom sugar
custom pure


In [18]:
class StaticMethod:
    def __init__(self, func):
        self.func = func
    def __get__(self, obj, obj_type = None):
        return self.func

class ClassMethod:
    def __init__(self, func):
        self.func = func
    def __get__(self, obj, obj_type = None):
        if obj_type is None: # конда вызывается от класса, то класс объекта возвращаем первым параметром
            obj_type = type(obj)
        def new_func(*args, **kwargs):
            return self.func(obj_type, *args, **kwargs)
        return new_func



### `__slots__`

На самом деле с помощью дескрипторов в Python реализовано очень много чего, и, например, есть такая конструкция __slots__, которая работает тоже с помощью дескрипторов. __slots__ позволяет вам определить класс, у которого есть жестко заданный набор атрибутов. Как вы знаете, когда мы создаем класс, у класса создается соответствующий словарь, в который мы записываем атрибуты, которые добавляются в объект. Очень часто это бывает излишне. У вас может быть огромное количество, например, объектов, и вы не хотите создавать каждый раз для каждого объекта словарь. Для этого приходит на помощь конструкция __slots__, которая вам позволяет жестко задать количество элементов, которые ваш класс может содержать. В данном случае мы говорим, что у нас в нашем классе должен быть только атрибут anakin. Собственно, он при инициализации и создается. Если мы попытаемся добавить в наш класс, в наш объект какой-то еще один атрибут, у нас ничего не получится, потому что у нас нет, собственно, справочника, нет dict, в который мы это записываем. И __slots__ реализуется с помощью определения дескрипторов для каждого из атрибутов.

In [20]:
class Class:
    __slots__ = ['anakin']
    
    def __init__(self):
        self.anakin = 'the chosen one ;)'

obj = Class()
print(obj.anakin)
obj.luke = 'the chosen too'

the chosen one ;)


AttributeError: 'Class' object has no attribute 'luke'

# Метаклассы
Как вы уже знаете, всё в Python'е является объектом, и классы не исключение, а значит эти классы кто-то создаёт. Давайте определим класс с названием Class и его объект. Тип нашего объекта является Class, потому что Class создал наш объект. Однако, у класса тоже есть тип. Этот тип type, потому что type создал наш класс. В данном случае type является мета-классом. Он создаёт другие классы. Типом самого type, кстати, является он же сам. Это рекурсивное замыкание, которое реализовано с помощью С внутри. Очень важно понимать разницу между созданием и наследованием. В данном случае класс не является subclass'ом type. Type его создаёт, но класс не наследуется от него.

In [27]:
class Class:
    pass

obj = Class()

print(type(obj))
print(type(Class)) # type created our Class, type is metaclass
print(type(type))
print(isinstance(Class, type))
print(issubclass(Class, type))
print(issubclass(Class, object))

<class '__main__.Class'>
<class 'type'>
<class 'type'>
True
False
True


Чтобы понять, как вообще классы задаются, можно написать простую функцию, которая класс возвращает. В данном случае мы определяем функцию, которая возвращает какой-то новый класс — dummy_factory. Классы можно создавать на лету. В данном случае мы создаём два разных объекта и просто их возвращаем. 

In [29]:
def dummy_factory():
    class Class:
        pass
    return Class

Dummy = dummy_factory()
print(Dummy() is Dummy())

False


Однако, на самом деле, Python работает, конечно, не так. Для создания классов используется мета-класс type, и вы можете на лету создать type, просто вызвав его и передав название класса. В данном случае мы создаём класс NewClass без родителей и без каких-то атрибутов. NewClass действительно является классическим классом. Вы можете его вывести, можете создать какой-то объект этого класса. Это настоящий класс, мы создали его на лету без литерала class.

In [31]:
NewClass = type('NewClass', (), {})
print(NewClass)
print(NewClass())

<class '__main__.NewClass'>


Однако, чаще всего классы создаются всё-таки по-другому. Они создаются с помощью мета-классов, и в данном случае давайте определим свой собственный мета-класс, который будет управлять поведением при создании класса. Мы определим класс Meta. Для того чтобы он бы мета-классом, он должен наследоваться от другого мета-класса. В данном случае это мета-класс type, базовый мета-класс. И, как вы уже знаете, метод _new_ управляет поведением при создании объекта. В данном случае объектом является другой класс, поэтому мы можем изменять поведение при создании другого класса. Метод _new_ принимает название класса, его родителей и какие-то атрибуты. Мы можем определить какой-то новый класс A и указать, что его мета-классом является наш мета-класс. Именно этот мета-класс и будет управлять поведением при создании нового класса. При определении class у нас вызывается мета-класс и функция _new_, метод _new_. Мы выводим, что у нас наш класс создаётся. Записываем в какой-то атрибут нашего класса, в данном случае в атрибут class_id, значение. Таким образом, мы можем переопределить поведение при создании класса. Например, добавить ему какой-то атрибут или сделать что-нибудь другое.

In [34]:
class Meta(type):
    def __new__(cls, name, parents, attrs):
        print('Creating {}'.format(name))
        if 'class_id' not in attrs:
            attrs['class_id'] = name.lower()
        return super().__new__(cls, name, parents, attrs)
    
class A(metaclass = Meta):  # here Meta will manage creation of class A
    pass

print('A.class_id: "{}"'.format(A.class_id))

Creating A
A.class_id: "a"


Например, мы можем определить мета-класс, который переопределяет функцию _init_, и в данном случае наш мета-класс будет логировать, запоминать все созданные подклассы. Давайте определим функцию _init_, которая будет вызываться при инициализации нашего объекта. В данном случае нашим объектом является класс. При инициализации класса у нас она будет вызываться. Наш _init_ принимает те же самые аргументы, однако, делает немного другое. Он записывает свой собственный атрибут значения созданных классов. В данном случае, у нас вначале создаётся класс Base, мета-классом которого является Meta, и у него создаётся registry, в который мы потом будем записывать все его подклассы. Каждый раз, когда у нас создаётся какой-то класс, который наследуется от Base, мы записываем в наш registry соответствующее значение, то есть название созданного класса и ссылку на него, то есть объект class. И мы можем вывести теперь все подклассы нашего Base, просто обратившись к registry, ну или написав обращение к методу subclasses.

In [35]:
class Meta(type):
    def __init__(cls, name, bases, attrs):
        print('Initializing - {}'.format(name))
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:    
            cls.registry[name.lower()] = cls
        super().__init__(name, bases, attrs)
        
class Base(metaclass = Meta):
    pass

class A(Base):
    pass

class B(Base): 
    pass

Initializing - Base
Initializing - A
Initializing - B


In [37]:
print(Base.registry)
print(Base.__subclasses__())

{'a': <class '__main__.A'>, 'b': <class '__main__.B'>}
[<class '__main__.A'>, <class '__main__.B'>]


### Абстрактные методы
В Python'е есть абстрактные методы, вы можете их использовать с помощью стандартной библиотеки abc. В данном случае здесь также работают мета-классы и мы можем определить абстрактный какой-то класс с методом abstractmethod. О чём говорит наш декоратор abstractmethod? Что у нас не получится создать какой-то класс, не определив этот метод. То есть у нас метод абстрактный и мы обязаны его переопределить в классе, который наследуется от нашего класса. 

In [38]:
from abc import ABCMeta, abstractmethod

class Sender(metaclass = ABCMeta):
    @abstractmethod
    def send(self):
        """Do something"""
        
class Child(Sender):
    pass

Child()

TypeError: Can't instantiate abstract class Child with abstract methods send

In [39]:
class Child(Sender):
    def send(self):
        print('Sending...')
        
Child()

Ну и, на самом деле, абстрактные методы используются в Python'е довольно редко, чаще всего вызывается просто исключение NotImplementedError, которое говорит программисту о том, что этот метод нужно реализовать.

In [ ]:
class PythonWay:
    def send(self):
        raise NotImplementedError

### Programming Assignment: Дескриптор с комиссией
Часто при зачислении каких-то средств на счет с нас берут комиссию. Давайте реализуем похожий механизм с помощью дескрипторов. Напишите дескриптор Value, который будет использоваться в нашем классе Account.

У аккаунта будет атрибут commission. Именно эту коммиссию и нужно вычитать при присваивании значений в amount.



In [43]:
class Value:
    def __init__(self, amount):
        self.amount = amount
    def __set__(self, obj, value):
        self.amount = value * (1 - obj.comission)
    def __get__(self, obj, obj_type):
        return self.amount
    
class Account:
    amount = Value(100)
    def __init__(self, comission):
        self.comission = comission
        
bobs_account = Account(0.1)
bobs_account.amount = 100
print(bobs_account.amount)

90.0


# Отладка

Мы с вами разберём классический механизм отладки с помощью Python Debugger'а и делать мы это будем на примере. Мы напишем программу, которая принимает на вход сайт, URL сайта и какую-то строчку, и ищет в коде сайта эту строчку и считает, сколько раз там встретилась эта строка. Давайте напишем нашу программу и попробуем её отладить

In [ ]:
import requests
import re

def main(site_url, substring):
    import pdb
    pdb.set_trace()
    
    site_code = get_site_code(site_url)
    matching_substrings = get_matching_substrings(site_code, substring)
    print('"{}" found {} times in {}'.format(
        substring, len(matching_substrings), site_url))
    
def get_matching_substrings(source, substring):
    return re.findall(substring, source)
    
def get_site_code(site_url):
    if not site_url.startswith('http'):
        site_url = 'http://' + site_url
    return requests.get(site_url).text

main('mail.ru', 'script')

> <ipython-input-9-ab52830b0998>(8)main()
      6     pdb.set_trace()
      7 
----> 8     site_code = get_site_code(site_url)
      9     matching_substrings = get_matching_substrings(site_code, substring)
     10     print('"{}" found {} times in {}'.format(

ipdb> b
ipdb> b10
*** NameError: name 'b10' is not defined
ipdb> b 10
Breakpoint 2 at <ipython-input-9-ab52830b0998>:10
ipdb> cont
> <ipython-input-9-ab52830b0998>(10)main()
      8     site_code = get_site_code(site_url)
      9     matching_substrings = get_matching_substrings(site_code, substring)
2--> 10     print('"{}" found {} times in {}'.format(
     11         substring, len(matching_substrings), site_url))
     12 



И давайте вызовем нашу функцию. В этот момент уже начнётся процесс отладки, потому что скорее всего сразу она не заработает. И, например, не заработает она потому, что мы не передали в функцию main параметра. Итак, процесс отладки уже начат, мы смотрим на исключения, читаем, что нам исключение это говорит. Давайте исправим это, передадим туда сайт mail.ru и script, допустим, поскольку script будем искать в сайте mail.ru. Давайте запустим и посмотрим, что у нас не работает дальше. Итак, у нас какая-то проблема в get_matching_substrings очевидно. Совершенно непонятно, что же случилось. По этой ошибке нельзя однозначно сказать, почему наша программа не работает. В данном случае на помощь нам может прийти отладчик.

В терминале для запуска дебаггера:
    
python3 -m pdb wc_web.py

Команды:
- ll - выход на точку, где определен pdb.set_trace()
- step/s - вход внутрь функции
- next/n - проход функции без захода
- b - показать брейкпоинты
- b 12 - поставить брейпоинт на строку 12
- cont/c - продолжить после остановки
- help - показать команды

## Тестирование

Мы с вами разберем наиболее популярный, наиболее распространенный вид тестирования - это unit тестирование. Unit тесты призваны протестировать какую-то небольшую функциональность, функцию, класс или модуль, посмотреть корректно ли он работает. Вы можете написать unit тесты к вашем классу, чтобы проверять все ли он делает корректно. Чтобы определить свой unittest можно воспользоваться стандартной библиотекой модулей minitest и определить свой класс, который наследуется от testcase'а из модуля unittest. Дальше вы можете определить функции, которые, собственно, и будут являться тестами. Каждая функция, которая начинается с test и нижнего подчеркивания, является тестом и внутри этого теста вы можете проверить какие-то условия. В данном случае мы можем проверить правильно ли у нас приводится тип в случае int'a и float'а, или, например, корректно ли у нас работает функция get у пустого словаря. Делается это с помощью методов testcase'а. Их довольно много - есть assertEqual, assertIsNone, assertRaises и так далее. Вы можете посмотреть про это в документации. Все они делают одно - они проверяют корректно ли работает выражение, корректно ли вызывается функция и так далее. Чтобы запустить тесты можно воспользоваться консолью. Еще чаще тесты запускает какая-то автоматическая система сборки или тестирования, или, например, ваше IDE может запускать тесты

Resources:
    
- https://docs.python.org/3/library/unittest.html
- https://docs.python.org/3/library/unittest.mock.html
- https://docs.python.org/3/library/unittest.mock-examples.html

In [ ]:
# test_python.py

import unittest

class TestPython(unittest.TestCase):
    def test_float_to_int_coercion(self):
        self.assertEqual(1, int(1.0))
    def test_get_empty_dict(self):
        self.assertIsNone({}.get('key'))
    def test_trueness(self):
        self.assertTrue(bool(10))

Запуск из терминала:
  
python3 -m unittest test_python.py

In [ ]:
# test_division.py

import unittest

class TestDivision(unittest.TestCase):
    def test_integer_division(self):
        self.assertIs(10 / 5, 2)

### Пример

Давайте посмотрим на конкретный пример и напишем свой собственный класс, который попробуем протестировать. Класс будет называться Asteroid и он призван помочь нам работать с открытым api NASA по астероидам и каким-то телам, которые летают вокруг Земли

In [2]:
import requests

class Asteroid:
    BASE_API_URL = 'https://api.nasa.gov/neo/rest/v1/neo/{}?api_key=DEMO_KEY'
    
    def __init__(self, spk_id):
        self.api_url = self.BASE_API_URL.format(spk_id)
    
    def get_data(self):
        return requests.get(self.api_url).json()
    
    @property
    def name(self):
        return self.get_data()['name']
    
    @property
    def diameter(self):
        return int(self.get_data()['estimated_diameter']['meters']['estimated_diameter_max'])
    

Давайте протестируем наш класс и посмотрим, корректно ли работает наша функция name и наша функция diameter. Однако, вы можете заметить, что здесь есть некоторая тонкость. Каждый раз, когда мы будем запускать тесты, у нас наш класс, testcase будет ходить в Интернет, потому что у нас запускается функция get_data. Это не всегда будет работать, потому что мы можем запускать наши тесты, например, в окружении, в котором нет Интернета, или Интернет медленный, или мы экономим трафик. Точно то же самое можно сказать про работу с Сетью в принципе или про работу с какими-то другими ресурсами, например, с диском. Возможно мы не хотим загружать диск. Что же сделать в таком случае? На помощь нам придет несколько механизмов, о которых мы поговорим прямо сейчас.

In [3]:
apophis = Asteroid(2099942)
# https://api.nasa.gov/neo/rest/v1/neo/2099942?api_key=DEMO_KEY

print(f'Name: {apophis.name}')
print(f'Diameter: {apophis.diameter} m')

Name: 99942 Apophis (2004 MN4)
Diameter: 682 m


Итак, во-первых, мы опять же определяем класс, который наследуется от testcase'а и определяем новую функцию setUp, с который вы еще не знакомы. Функция setUp призвана, как и соответствует ее название, "засетапить" окружение, которое будет работать во время исполнения тестовой функции. Таким образом, если нам нужно работать, например, с объектом Asteroid'а, мы можем в начале исполнения каждой функции создавать этот объект Asteroid'а, чтобы не дублировать этот код каждый раз в начале наших тестовых функций, или мы можем создавать другие какие-то объекты или как-то наши данные готовить. Существует симметричный метод, который называется tearDown, который позволяет закрывать какие-то ресурсы, удалять объекты в конце каждой тестовой функции. Итак, давайте напишем наши две тестовые функции, которые будут проверять test_name, то есть diameter. Однако, что если мы тестируем наши функции в окружении без Интернета, как я вам уже говорил. На помощь нам может прийти механизм mock off и модуль unittest.mock, который позволяет подменять какую-то функциональность, подменять какие-то функции другими. Таким образом, мы можем на самом деле не ходить в Интернет, а можем, например, читать информацию из файла. Я заранее скачал данные об астероиде Apophis в специальную фикстуру, текстовый файл. Это просто точно то же самое, что и возвращает наш api, только оно лежит в файле. Мы подменим функцию, которая идет в Интернет get_data функцией, которая просто читает из файла. Таким образом мы не будет ходить в Интернет во время тестов. Делается это с помощью декоратора patch довольно просто. Мы можем проверять внутри нашей тестовой функции определенные условия. В данном случае мы проверяем действительно ли имя астероида, которое мы распарсили, в данном случае из файла, действительно оно корректно, действительно ли оно Apophis и проверять размер нашего астероида - действительно ли он равен почти 700 метрам

In [ ]:
# test_asteroid.py

import json
import unittest
from unittest.mock import patch
from asteroid import Asteroid

class TestAsteroid(unittest.TestCase):
    def setUp(self):
        self.asteroid = Asteroid(2099942)
        
    def mocked_get_data(self):
        with open('data/apophis_fixture.txt') as f:
            return json.loads(f.read())
        
    @patch('asteroid.Asteroid.get_data', mocked_get_data)
    def test_name(self):
        self.assertEqual(
            self.asteroid.name, '99942 Apophis (2004 MN4)'
        )
        
    @patch('asteroid.Asteroid.get_data', mocked_get_data)
    def test_diameter(self):
        self.assertEqual(self.asteroid.diameter, 682)